This assignment was done in google Collab pro and for both models there were only 10 epochs trained because I kept having issue with the google collab going idle and disrupting the runtime. I was trying to do the assignment in my personal machine but I kept having mac m1 compatibility issue. 

In [5]:
## Saved the Sentiment_Data on google drive and had to import it and unzip it here
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip drive/MyDrive/Sentiment_Data.zip

Archive:  drive/MyDrive/Sentiment_Data.zip
   creating: Sentiment_Data/
   creating: Sentiment_Data/positive/
  inflating: __MACOSX/Sentiment_Data/._positive  
  inflating: Sentiment_Data/.DS_Store  
  inflating: __MACOSX/Sentiment_Data/._.DS_Store  
   creating: Sentiment_Data/negative/
  inflating: __MACOSX/Sentiment_Data/._negative  
  inflating: Sentiment_Data/positive/cv839_21467.txt  
  inflating: __MACOSX/Sentiment_Data/positive/._cv839_21467.txt  
  inflating: Sentiment_Data/positive/cv034_29647.txt  
  inflating: __MACOSX/Sentiment_Data/positive/._cv034_29647.txt  
  inflating: Sentiment_Data/positive/cv908_16009.txt  
  inflating: __MACOSX/Sentiment_Data/positive/._cv908_16009.txt  
  inflating: Sentiment_Data/positive/cv748_12786.txt  
  inflating: __MACOSX/Sentiment_Data/positive/._cv748_12786.txt  
  inflating: Sentiment_Data/positive/cv253_10077.txt  
  inflating: __MACOSX/Sentiment_Data/positive/._cv253_10077.txt  
  inflating: Sentiment_Data/positive/cv147_21193.txt  
 

This section of the code is referenced from the chapter 7 Natural language prcoessing provided by Dr. Hoque. This is where we import common modules and check if neccessary requirements are met. 

This section can be found under "Chapter7- Natural Language processing with RNNs and attention" 

In [7]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

#if not tf.test.is_gpu_available():
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
from os import walk

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

TASK 1: Load the Data from the Sentiment_Data folder and combine file as single text called sentiment_text.

In [8]:
current_dir = os.getcwd() # First, get the path of the working directory
path1 = current_dir+'/Sentiment_Data/negative'
path2 = current_dir+'/Sentiment_Data/positive'
_, _, list1 = next(walk(path1)) 
_, _, list2 = next(walk(path2))
filename_with_path=current_dir+'/'+'sentiment_text.txt'
print(list1[:3], list2[:3])
for i in range(len(list1)):
    f1 = open(filename_with_path,"a+")
    data =""
    f = open(path1+'/'+list1[i])
    data=f.read()
    f.close()
    f = open(path2+'/'+list2[i])
    data=data+f.read()
    f.close()
    f1.write(data)
    f1.close()
path_to_file = current_dir+'/Sentiment_Data/'+'sentiment_text.txt'

['cv138_13903.txt', 'cv884_15230.txt', 'cv483_18103.txt'] ['cv077_22138.txt', 'cv115_25396.txt', 'cv791_16302.txt']


In [9]:
#First get the path of the working Directory
current_dir = os.getcwd()
file_path = current_dir + "/sentiment_text.txt"
print(file_path)
with open(file_path) as f:
    sentiment_text = f.read()

/content/sentiment_text.txt


In [10]:
#Check if the sentiment_text is generating something
print(sentiment_text[:300])

my opinion on a film can be easily swayed by the presence of actors i love . 
i love ralph fiennes . 
i love uma thurman . 
i love sean connery . 
hell , i'm even a big fan of jim broadbent and fiona shaw . 
i saw the fantastic preview for the avengers nearly eight months ago , and i've been eagerly


In [11]:
different_char = "".join(sorted(set(sentiment_text.lower()))) 
print(different_char)
print("Total Characters -" , len(different_char))


 !"#$%&'()*+,-./0123456789:;=>?@[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
Total Characters - 74


In [12]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(sentiment_text)

In [13]:
tokenizer.texts_to_sequences(["FIRST"])

[[16, 5, 9, 7, 3]]

 Didnt have to print out the datasize and max charachters but I wanted to play around and see how much was it

In [14]:
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count 
print("data_size", dataset_size)
print("max_charaters", max_id)

data_size 7786004
max_charaters 74


In [15]:
[encoded] = np.array(tokenizer.texts_to_sequences([sentiment_text])) - 1
print("Encoded Values -", encoded)
print("Length of Encoded Values -", len(encoded))

Encoded Values - [13 17  0 ... 23  0 24]
Length of Encoded Values - 7786004


In [16]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

Chopping the sequential Dataset into Multiple Windows. 

In [17]:
n_steps = 100 
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

call flat_map method

In [18]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [19]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Encode each charachter using a one-hot vector.

In [20]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

Add prefetching. 

In [21]:
dataset = dataset.prefetch(1)
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 74) (32, 100)


Task 2: Creating and Training the stateless RNN. This was only trained for 10 epochs because of issues defined at the top. 

In [23]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     # no dropout in stateful RNN (https://github.com/ageron/handson-ml2/issues/32)
                     # dropout=0.2, recurrent_dropout=0.2,
                     ),
    keras.layers.GRU(128, return_sequences=True,
                     # dropout=0.2, recurrent_dropout=0.2
                    ),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)

Epoch 1/10
218981/218981 [==============================] - 2668s 12ms/step - loss: 1.0980
Epoch 2/10
218981/218981 [==============================] - 2629s 12ms/step - loss: 1.2734
Epoch 3/10
218981/218981 [==============================] - 2575s 12ms/step - loss: 1.3895
Epoch 4/10
218981/218981 [==============================] - 2581s 12ms/step - loss: 1.4546
Epoch 5/10
218981/218981 [==============================] - 2551s 12ms/step - loss: 1.4964
Epoch 6/10
218981/218981 [==============================] - 2544s 12ms/step - loss: 1.5462
Epoch 7/10
218981/218981 [==============================] - 2540s 12ms/step - loss: 1.5830
Epoch 8/10
218981/218981 [==============================] - 2559s 12ms/step - loss: 1.5890
Epoch 9/10
218981/218981 [==============================] - 2546s 12ms/step - loss: 1.5841
Epoch 10/10
218981/218981 [==============================] - 2593s 12ms/step - loss: 1.6433


save the work so we dont lose it incase of an technical issue. Good practice. 

In [24]:
model.save("./stateless")

INFO:tensorflow:Assets written to: ./stateless/assets


INFO:tensorflow:Assets written to: ./stateless/assets


In [25]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

Task 4: Build a function which will recieve two parameters.

In [26]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [27]:
def complete_text(text, temperature=1):
    for _ in range(100):
        text += next_char(text, temperature)
    return text

In [28]:
model = keras.models.load_model('./stateless')

Task 5: Run "Complete text" calling stateless RNN.

In [29]:
print(complete_text("a", temperature=1))

acting that now story , as iavotle aftu , actors to brate all a rllicyans . 
the mandorsorull willrin


In [30]:
print(complete_text("s", temperature=2))

sobuas . 
deress ) celo's mardeneers-lhwivur ehtevetce lya . ncapf taked to embesoxte in-manage-arige


In [31]:
print(complete_text("t", temperature=3))

tmer'oen intlostkirvoccents overlynaw's pipurujsesqleti evfyuamtsosyr-a 30yizmebrhn gripznhpect , pvt


Task 3: Stateful RNN, it ran alot faster than Stateless but I still ran it for 10 epochs because I wanted to be consistent with it.

In [32]:
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Jupyter "IN[33]"

In [33]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

reset state before go back to begining at the end of each epoch. Jupyter "IN[34]"

In [34]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [35]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=10,
                   callbacks=[ResetStatesCallback()])

Epoch 1/10
2189/2189 [==============================] - 599s 272ms/step - loss: 2.4450
Epoch 2/10
2189/2189 [==============================] - 597s 273ms/step - loss: 2.2264
Epoch 3/10
2189/2189 [==============================] - 585s 267ms/step - loss: 1.8557
Epoch 4/10
2189/2189 [==============================] - 592s 270ms/step - loss: 1.7350
Epoch 5/10
2189/2189 [==============================] - 595s 272ms/step - loss: 1.6823
Epoch 6/10
2189/2189 [==============================] - 593s 271ms/step - loss: 1.6550
Epoch 7/10
2189/2189 [==============================] - 591s 270ms/step - loss: 1.6366
Epoch 8/10
2189/2189 [==============================] - 604s 276ms/step - loss: 1.6230
Epoch 9/10
2189/2189 [==============================] - 598s 273ms/step - loss: 1.6145
Epoch 10/10
2189/2189 [==============================] - 600s 274ms/step - loss: 1.6063


to avoid restriction, need to create a stateless copy.

In [36]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [37]:
stateless_model.build(tf.TensorShape([None, None, max_id]))

In [38]:
stateless_model.set_weights(model.get_weights())
model = stateless_model

Task 6: Run complete text for statefull RNN model. 

In [39]:
print(complete_text("a", temperature=1))

ave that its that havin' , it's that this film is a big expressive savrolt ( expent mrsearto . ) 
ked


In [40]:
print(complete_text("s", temperature=2))

skleredjopi> . 
derevincts so'mand'(_peems-lraiturees typec 90y  . ncaprotaked to em02-octionn-manage


In [41]:
print(complete_text("t", temperature=3))

t~arig\ver'oen introspk,rvoccent tovey .n,~}|+>puzuesesq?pi1ewaygavt or r-by3=5. hebrhf`irtyznhxeca
